In [5]:
!sudo apt install poppler-utils

[sudo] password for aliffatur: 
sudo: a password is required
^C


In [ ]:
!sudo apt install poppler-utils
%pip install torch pdf2image pillow sentence-transformers faiss-cpu surya-ocr

  Using cached scikit_learn-1.7.2-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (11 kB)
  Using cached scipy-1.16.2-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (62 kB)
  Using cached joblib-1.5.2-py3-none-any.whl.metadata (5.6 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.0/488.0 kB 235.2 kB/s eta 0:00:00a 0:00:01
Using cached scikit_learn-1.7.2-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (9.5 MB)
Using cached scipy-1.16.2-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (35.7 MB)
Using cached joblib-1.5.2-py3-none-any.whl (308 kB)
Using cached threadpoolctl-3.6.0-py3-none-any.whl (18 kB)
Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
import json
from pdf2image import convert_from_path
import numpy as np
from sentence_transformers import SentenceTransformer
import faiss

from surya.foundation import FoundationPredictor
from surya.detection import DetectionPredictor
from surya.recognition import RecognitionPredictor
from surya.layout import LayoutPredictor
from surya.settings import settings


# ===== KONFIGURASI =====
PDF_PATH = "data/data_raw/handbook-of-pharmaceutical-excipients-6th-ed-A.pdf"
OUTPUT_JSON = "dokumen_faiss_ready.json"
OUTPUT_FAISS = "faiss_index.bin"
EMBED_MODEL = "sentence-transformers/all-MiniLM-L6-v2"
# ========================


def extract_pdf_layout_ocr(pdf_path):
    """Jalankan layout analysis + OCR multilayout dengan urutan baca (reading order)"""
    print("📄 Mengonversi PDF ke gambar...")
    images = convert_from_path(pdf_path, dpi=300)

    print("⚙️ Memuat model Surya (foundation, layout, deteksi, OCR)...")
    foundation = FoundationPredictor()
    detection = DetectionPredictor()
    recognition = RecognitionPredictor(foundation)
    layout = LayoutPredictor(foundation)

    all_data = []

    for page_num, img in enumerate(images):
        print(f"🔍 Memproses halaman {page_num + 1}...")

        # Layout analysis
        layout_result = layout([img])[0]

        # OCR detection + recognition
        rec_result = recognition([img], det_predictor=detection)[0]

        page_data = []
        # Iterasi elemen layout
        for element in getattr(layout_result, "elements", []):
            bbox = element.bbox
            label = element.label if hasattr(element, "label") else "Text"
            position = getattr(element, "reading_order", None)

            # Ambil teks di dalam bounding box ini
            contained_texts = []
            for line in getattr(rec_result, "text_lines", []):
                line_bbox = line.bbox
                if (
                    line_bbox[0] >= bbox[0]
                    and line_bbox[1] >= bbox[1]
                    and line_bbox[2] <= bbox[2]
                    and line_bbox[3] <= bbox[3]
                ):
                    contained_texts.append(line.text)

            combined_text = " ".join(contained_texts).strip()
            if combined_text:
                page_data.append({
                    "page": page_num + 1,
                    "layout_label": label,
                    "reading_order": position,
                    "bbox": bbox,
                    "text": combined_text
                })

        # Urutkan jika ada reading order
        page_data_sorted = sorted(page_data, key=lambda x: x.get("reading_order", 9999))
        all_data.extend(page_data_sorted)

    print(f"✅ OCR dan layout extraction selesai ({len(all_data)} blok teks)")
    return all_data


def save_json(data, output_path):
    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=2)
    print(f"📁 Hasil disimpan ke {output_path}")


def build_faiss_index(json_data, output_path, embed_model):
    print("⚙️ Membuat embedding FAISS...")
    model = SentenceTransformer(embed_model)
    texts = [item["text"] for item in json_data if item.get("text")]
    embeddings = model.encode(texts, show_progress_bar=True)
    index = faiss.IndexFlatL2(embeddings.shape[1])
    index.add(np.array(embeddings, dtype="float32"))
    faiss.write_index(index, output_path)
    print(f"✅ FAISS index disimpan ke {output_path}")
    return index


if __name__ == "__main__":
    data = extract_pdf_layout_ocr(PDF_PATH)
    save_json(data, OUTPUT_JSON)
    build_faiss_index(data, OUTPUT_FAISS, EMBED_MODEL)
    print("🎯 Semua proses selesai!")


/home/aliffatur/coding/kalbe-cdt-intern/explore_llm/cv_to_vector_db/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


📄 Mengonversi PDF ke gambar...
⚙️ Memuat model Surya (foundation, layout, deteksi, OCR)...
🔍 Memproses halaman 1...


Recognizing Text: 100%|██████████| 140/140 [00:07<00:00, 18.12it/s]


🔍 Memproses halaman 2...


Recognizing Text: 100%|██████████| 140/140 [00:10<00:00, 13.51it/s]


...

🔍 Memproses halaman 52...


Recognizing Text: 100%|██████████| 11/11 [00:08<00:00,  1.35it/s]


✅ OCR dan layout extraction selesai (0 blok teks)
📁 Hasil disimpan ke dokumen_faiss_ready.json
⚙️ Membuat embedding FAISS...


Batches: 0it [00:00, ?it/s]


IndexError: tuple index out of range